# Preprocesamiento

En esta etapa generamos las imágenes que utilizará el modelo para el entrenamiento y la predicción. 

Primero se generan las imágenes de n-bandas. Es decir las imágenes que no solo poseen los canales RGB sino además información de otras bandas como el infrarrojo cercano (NIR near-infra-red),  o mismo, una combinación de bandas. En esta etapa consideramos toda la información que podría ser sensible a la detección del objeto de interés.

Luego, utilizando las imágenes generadas en el paso anterior y la información vectorial de la ubicación de los basurales, generamos el dataset utilizado luego por el modelo de ML. Este dataset se genera con el formato necesario para el modelo.



## Preprocesamiento para las imagenes de entrenamiento


Descargamos las librerías glob y os para la navegación de archivos, y tqdm para visualizar el progreso del procesamiento.

In [2]:
import glob
import os
from tqdm import tqdm

### Descarga de imagenes 

In [ ]:
#!mkdir ../images_s2/

In [ ]:
#!gsutil -m cp -r gs://dym-datasets-public/basurales/train_6D/ ../images_s2/
#!gsutil -m cp -r gs://dym-datasets-public/basurales/predict_6D/ ../images_s2/

### Setup


In [3]:
PATHS = [#'../images_s2/train_6D/2021-02-01_2021-02-28/*.tif',
         '../images_s2/predict_6D/2021-02-01_2021-02-28/*.tif',
         ]


### Compute  NDSW index


Generamos la banda Normalized Difference ShortWave (NDSW) como la diferencia normalizada entre las dos bandas del infrarrojo de onda corta, es decir NDSW = (SWIR12-SWIR11)/(SWIR12 + SWIR11 ) . Esta diferencia de bandas mostró ser sensible a la emisión de metano en estudios previos.  Dicho componente puede ser un indicador de la presencia de un basural con residuos biológicos.


In [4]:

# para cada imagen en los grupos de entrenamiento prediccion y control, generamos la banda NDSW
for path in PATHS:
   
    images = glob.glob(path) #ruta a cada imagen 
    
    for image in tqdm(images):
        
        #skip resulting files 
        if "_diffSW" in image: continue
        if "_ori_tmp" in image: continue
        if "rgb-nir-sw-diffSW"  in image: continue
            
        fname, ext = os.path.splitext(image)
        
        #--------------------------------
        #  Generacion de la banda NDSW  
        #--------------------------------
        
        ndsw_file = str(fname+'_diffSW.tif')
        #usamos gdal_calc para calcular la nueva banda usando la banda 5 y 6 de cada imagen
        !gdal_calc.py -A $image --A_band=5 -B $image --B_band=6 --calc="(A-B)/(A+B)" --outfile=$ndsw_file 
        
        #------------------------------------------
        #  Generacion de la bandas RGB - NIR -SWIR  
        #------------------------------------------
        
        img_5bands = str(fname+'_ori_tmp.tif')
        #usamos gdal_translate quedarnos con las 5 primeras bandas de cada imagen
        !gdal_translate -b 1 -b 2 -b 3 -b 4 -b 5 $image $img_5bands

        #----------------------------------------------
        #  Generacion de imagen RGB - NIR -SWIR -NDSW 
        #----------------------------------------------        
        
        rgb_nir_swir_diffSW_fname = str(fname+'_rgb-nir-sw-diffSW.tif')
        #usamos otbcli_ConcatenateImages para concatenar las imagenes generadas en el paso anterior quedarnos con las 5 primeras bandas de cada imagen
        !bash -c "source /opt/OTB/otbenv.profile; otbcli_ConcatenateImages -il  $img_5bands $ndsw_file -out $rgb_nir_swir_diffSW_fname"


  0%|                                                                         | 0/88 [00:00<?, ?it/s]

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 

Input file size is 13568, 13568
0...10...20...30...40...50...60...70...80...90...100 - done.
2022-02-02 16:13:08 (INFO) ConcatenateImages: Default RAM limit for OTB is 256 MB
2022-02-02 16:13:08 (INFO) ConcatenateImages: GDAL maximum cache size is 749 MB
2022-02-02 16:13:08 (INFO) ConcatenateImages: OTB will use at most 4 threads
2022-02-02 16:13:09 (INFO): Estimated memory for full processing: 16853.8MB (avail.: 256 MB), optimal image partitioning: 66 blocks
2022-02-02 16:13:09 (INFO): File ../images_s2/predict_6D/2021-02-01_2021-02-28/0000067840-0000013568_rgb-nir-sw-diffSW.tif will be written in 67 blocks of 13568x203 pixels
Writing ../images_s2/predict_6D/2021-02-01_2021-02-28/0000067840-0000013568_rgb-nir-sw-diffSW.tif...: 100% [**************************************************] (26s)


  5%|██▉                                                              | 4/88 [02:08<44:51, 32.05s/it]

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 9.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 10.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 11.. 12.. 12.. 12.. 12.. 12.. 12.. 12.. 12.. 12.. 1

...

Input file size is 13568, 13568
0...10...20...30...40...50...60...70...80...90...100 - done.
2022-02-02 17:36:22 (INFO) ConcatenateImages: Default RAM limit for OTB is 256 MB
2022-02-02 17:36:22 (INFO) ConcatenateImages: GDAL maximum cache size is 749 MB
2022-02-02 17:36:22 (INFO) ConcatenateImages: OTB will use at most 4 threads
2022-02-02 17:36:22 (INFO): Estimated memory for full processing: 16853.8MB (avail.: 256 MB), optimal image partitioning: 66 blocks
2022-02-02 17:36:22 (INFO): File ../images_s2/predict_6D/2021-02-01_2021-02-28/0000000000-0000000000_rgb-nir-sw-diffSW.tif will be written in 67 blocks of 13568x203 pixels
Writing ../images_s2/predict_6D/2021-02-01_2021-02-28/0000000000-0000000000_rgb-nir-sw-diffSW.tif...: 100% [**************************************************] (25s)


 97%|███████████████████████████████████████████████████████████▉  | 85/88 [1:25:20<04:21, 87.12s/it]

0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 1.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 2.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 3.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 4.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 5.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 6.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 7.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 8.. 

Input file size is 13568, 13568
0...10...20...30...40...50...60...70...80...90...100 - done.
2022-02-02 17:39:09 (INFO) ConcatenateImages: Default RAM limit for OTB is 256 MB
2022-02-02 17:39:09 (INFO) ConcatenateImages: GDAL maximum cache size is 749 MB
2022-02-02 17:39:09 (INFO) ConcatenateImages: OTB will use at most 4 threads
2022-02-02 17:39:09 (INFO): Estimated memory for full processing: 16853.8MB (avail.: 256 MB), optimal image partitioning: 66 blocks
2022-02-02 17:39:09 (INFO): File ../images_s2/predict_6D/2021-02-01_2021-02-28/0000013568-0000000000_rgb-nir-sw-diffSW.tif will be written in 67 blocks of 13568x203 pixels
Writing ../images_s2/predict_6D/2021-02-01_2021-02-28/0000013568-0000000000_rgb-nir-sw-diffSW.tif...: 100% [**************************************************] (45s)


100%|██████████████████████████████████████████████████████████████| 88/88 [1:28:26<00:00, 60.30s/it]


## Generación del dataset de entrenamiento

El modelo de ML utiliza para entrenar un dataset compuesto por imágenes y máscaras binarias que delimitan el objeto de interés. Estas se generan vía la herramienta **satproc_extract_chips**. Para el caso del dataset de entrenamiento se generan imágenes y máscaras (utilizando las anotaciones de verdad de campo) y para el de predicción solo imágenes.

Para generar las imágenes de entrenamiento:

In [ ]:
!pip install pysatproc

In [ ]:
path_to_files = "../data/img/img_train/2021-02-01_2021-02-28/*_rgb-nir-sw-diffSW.tif"
output_folder = "../dataset/data_train/6D/100_30/"

vector_file = "../data/shp/gt/gt_basurales_4326.geojson"

size = 100
step_size = 30

In [ ]:
!satproc_extract_chips \
    $path_to_files \
    -o  $output_folder \
    --size $size \
    --step-size $step_size \
    --aoi $vector_file \
    --labels $vector_file \
    --label-property 'class' \
    --classes 'B' \
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100

###### Los argumentos:

* **El primer argumento** es la ruta a las imágenes 

* **o** es la ruta de destino 

Recomendamos que dicha ruta sea descriptiva, por ejemplo “data_train/600_600/ ” describe : Data_train → datos usados para entrenar; 400_400 → <tamaño de la imagen >_ <tamaño del step-size> (las imágenes son cuadradas)

* **size** tamaño de las imágenes resultantes (las imágenes son cuadradas) 
* **step-size** paso del proceso. Si *step-size* es igual que el *size* entonces no hay overlap en las imágenes. 

En ocaciones es útil para el entrenamiento generar los chips con un overlap de este modo tenemos más datos para entrenar. Pero en la predicción valor debe ser igual al tamaño que la imagen 

* **crs** epsg: le asigna un epsg a la imagen 

* **label-property** nombre del campo donde se define cada categoría (solo se usa para el entrenamiento) 

* **classes** nombres de las clases (como aparecen en el geojson), separados por espacios

* **aoi** ruta al archivo vectorial donde están definidas las localidades. Al definir una region de interés solo se procesan las imágenes que interceptan esas localidades.

* **rescale** lleva los valores de las bandas a 0-255

Este comando va a generar dos carpetas en la ruta de destino : “images” y “masks”. Los archivos de la primera van a ser de tipo Tiff de 3 bandas (rgb) y los de la segunda van a ser, también, de tipo Tiff pero de N bandas donde N representa el número de clases, en este caso sólo una. Y donde cada una de las bandas es una máscara binaria


## Generación del dataset de predicción


De modo similar repetimos el procedimiento. En este caso el unico archivo vectorial delimita las zonas de interés para la predicción. 

### dataset de predicción

In [2]:
#!satproc_extract_chips --help

In [6]:
!ls ../dataset/data_predict/6D/100_100/

images


In [3]:
path_to_files = "../images_s2/predict_6D/2021-02-01_2021-02-28/*_rgb-nir-sw-diffSW.tif"
output_folder = "../dataset/data_predict/6D/100_100/"

vector_file_aoi = "../data/shp/aoi-predict.geojson"

size = 100


!satproc_extract_chips \
    $path_to_files \
    -o $output_folder \
    --size $size \
    --step-size $size \
    --aoi $vector_file_aoi \
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100 \
    --skip-existing

Rasters:   0%|                                           | 0/42 [00:00<?, ?it/s]
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   0%| | 0/18496 [00:00<?, ?
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   1%| | 187/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   2%| | 420/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   4%| | 691/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   5%| | 963/18496 [00:00<00
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   7%| | 1256/18496 [00:00<0
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:   8%| | 1539/18496 [00:00<0
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:  10%|1| 1868/18496 [00:00<0
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:  12%|1| 2205/18496 [00:00<0
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:  14%|1| 2553/18496 [00:00<0
0000000000-0000000000_rgb-nir-sw-diffSW.tif windows:  16%|1| 2921/18496 [00:01<0
0000000000-0000000000_rgb-ni

0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  54%|5| 9931/18496 [00:26<0
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  54%|5| 9961/18496 [00:26<0
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  54%|5| 9991/18496 [00:26<0
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  54%|5| 10022/18496 [00:26<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  54%|5| 10052/18496 [00:26<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  55%|5| 10083/18496 [00:26<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  55%|5| 10113/18496 [00:26<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  55%|5| 10142/18496 [00:27<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  55%|5| 10173/18496 [00:27<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  55%|5| 10204/18496 [00:27<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  55%|5| 10234/18496 [00:27<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  55%|5| 10263/18496 [00:27<
0000054272-0000000000_rgb-ni

0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  85%|8| 15704/18496 [00:49<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  85%|8| 15735/18496 [00:49<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  85%|8| 15766/18496 [00:49<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  85%|8| 15796/18496 [00:49<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  86%|8| 15826/18496 [00:49<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  86%|8| 15856/18496 [00:49<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  86%|8| 15884/18496 [00:49<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  86%|8| 15914/18496 [00:50<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  86%|8| 15943/18496 [00:50<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  86%|8| 15974/18496 [00:50<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  87%|8| 16005/18496 [00:50<
0000054272-0000000000_rgb-nir-sw-diffSW.tif windows:  87%|8| 16036/18496 [00:50<
0000054272-0000000000_rgb-ni

0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  14%|1| 2669/18496 [00:10<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  15%|1| 2700/18496 [00:10<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  15%|1| 2730/18496 [00:10<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  15%|1| 2760/18496 [00:10<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  15%|1| 2791/18496 [00:10<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  15%|1| 2821/18496 [00:11<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  15%|1| 2850/18496 [00:11<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  16%|1| 2881/18496 [00:11<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  16%|1| 2911/18496 [00:11<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  16%|1| 2942/18496 [00:11<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  16%|1| 2971/18496 [00:11<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  16%|1| 2999/18496 [00:11<0
0000054272-0000013568_rgb-ni

0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  46%|4| 8424/18496 [00:33<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  46%|4| 8452/18496 [00:33<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  46%|4| 8482/18496 [00:33<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  46%|4| 8510/18496 [00:33<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  46%|4| 8538/18496 [00:33<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  46%|4| 8567/18496 [00:34<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  46%|4| 8596/18496 [00:34<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  47%|4| 8626/18496 [00:34<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  47%|4| 8654/18496 [00:34<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  47%|4| 8685/18496 [00:34<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  47%|4| 8716/18496 [00:34<0
0000054272-0000013568_rgb-nir-sw-diffSW.tif windows:  47%|4| 8746/18496 [00:34<0
0000054272-0000013568_rgb-ni

...

0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   0%| | 0/5304 [00:00<?, ?i
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   0%| | 6/5304 [00:00<01:30
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   1%| | 34/5304 [00:00<00:2
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   1%| | 62/5304 [00:00<00:2
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   2%| | 90/5304 [00:00<00:2
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   2%| | 119/5304 [00:00<00:
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   3%| | 145/5304 [00:00<00:
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   3%| | 173/5304 [00:00<00:
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   4%| | 201/5304 [00:00<00:
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   4%| | 229/5304 [00:00<00:
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   5%| | 257/5304 [00:01<00:
0000081408-0000013568_rgb-nir-sw-diffSW.tif windows:   5%| | 284/5304 [00:01<00:
0000081408-0000013568_rgb-ni

0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   3%| | 147/5304 [00:00<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   3%| | 177/5304 [00:00<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   4%| | 205/5304 [00:00<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   4%| | 233/5304 [00:00<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   5%| | 261/5304 [00:01<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   5%| | 288/5304 [00:01<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   6%| | 317/5304 [00:01<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   7%| | 345/5304 [00:01<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   7%| | 374/5304 [00:01<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   8%| | 402/5304 [00:01<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   8%| | 429/5304 [00:01<00:
0000081408-0000027136_rgb-nir-sw-diffSW.tif windows:   9%| | 457/5304 [00:01<00:
0000081408-0000027136_rgb-ni

0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   4%| | 223/5304 [00:00<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   5%| | 251/5304 [00:01<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   5%| | 277/5304 [00:01<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   6%| | 304/5304 [00:01<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   6%| | 331/5304 [00:01<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   7%| | 359/5304 [00:01<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   7%| | 388/5304 [00:01<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   8%| | 414/5304 [00:01<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   8%| | 441/5304 [00:01<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   9%| | 469/5304 [00:02<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:   9%| | 497/5304 [00:02<00:
0000081408-0000040704_rgb-nir-sw-diffSW.tif windows:  10%| | 524/5304 [00:02<00:
0000081408-0000040704_rgb-ni

0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:   6%| | 301/5304 [00:01<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:   6%| | 328/5304 [00:01<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:   7%| | 355/5304 [00:01<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:   7%| | 382/5304 [00:01<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:   8%| | 408/5304 [00:01<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:   8%| | 434/5304 [00:01<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:   9%| | 461/5304 [00:02<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:   9%| | 489/5304 [00:02<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:  10%| | 518/5304 [00:02<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:  10%|1| 545/5304 [00:02<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:  11%|1| 572/5304 [00:02<00:
0000081408-0000054272_rgb-nir-sw-diffSW.tif windows:  11%|1| 601/5304 [00:02<00:
0000081408-0000054272_rgb-ni

0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  13%|1| 467/3627 [00:01<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  14%|1| 496/3627 [00:01<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  14%|1| 525/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  15%|1| 553/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  16%|1| 582/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  17%|1| 611/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  18%|1| 640/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  18%|1| 668/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  19%|1| 698/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  20%|2| 727/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  21%|2| 755/3627 [00:02<00:
0000081408-0000067840_rgb-nir-sw-diffSW.tif windows:  22%|2| 784/3627 [00:03<00:
0000081408-0000067840_rgb-ni